<a href="https://colab.research.google.com/github/ninivehelen/AADC-DF/blob/master/exploracao_dados_sistec_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Este arquivo deve conter as funções

import pandas as pd
import numpy as np
import math
# Biblioteca para regex
import re
# Biblioteca para data
from datetime import datetime

REGEX = '^[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}$'
TAMANHO_CPF = 11

'''
    INICIO: ↓  Funções para limpeza de cada coluna  ↓
'''

"""
  Usar a variável "consultaMatriculaCiclo" que contem os dados carregados

  Para criar um padrão de analise, pode ter um ponteiro para varrer as colunas 
  e outro para varrer os registros. Assim, podemos passar para as funções
  a coluna contendo todos os registros e ela retorna o resumo do que foi 
  encontrato. Ex.:

                    posicaoColuna
                          ↓
  -------------------------------------------------------------------------
  co_unidade_ensino | co_simec | unidade_ensino | dt_deferimento_ue | ... |
  -------------------------------------------------------------------------
  23                | 9874     | SENAC          | 21/03/2009        |     |
  131               | 65467    | SENAI          | 22/01/2009        |     |
  45                | 321      | IFB            | 10/05/2011        |     | ← posicaoLinha
  78                | 11       | SENAC          | 03/03/2010        |     |
  77                | 7412     | SEST           | 15/04/2012        |     |
  ...
  -------------------------------------------------------------------------

  Portanto, passaremos para a função a coluna inteira dos dados a analisar e 
  a coluna inteira dos códigos únicos da linha, e a função percorrerá
  todas linhas/registros analisando e fazendo os processos necessários.

  A documentação das funções devem seguir o padrão do Pandas Docstring Guide:
  > https://pandas.pydata.org/docs/development/contributing_docstring.html

"""

def valida_inteiro_nao_nulo(df_colunas):
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
            1.  co_unidade_ensino
            9.  co_ciclo_matricula
            14. co_curso
            17. co_tipo_curso
            19. co_tipo_nivel
            24. carga_horario
            41. ano (Campo removido)

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (1,9,14,17,19,24)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_co_unidade_ensino( df_colunas )
        Out:
        ----------------------------------------------------------------------------------
        codigo_universal_linha | coluna                 | tipo_inconsistencia | categoria
        ----------------------------------------------------------------------------------
        3                      | co_unidade_ensino      | NaN                 |  
        150                    | co_ciclo_matricula     | str                 |
                               | co_curso               |                     |
                               | co_tipo_curso          |                     |
                               | co_tipo_nivel          |                     |  
                               | carga_horario          |                     |  
                               | ano (Campo removido)   |                     |
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
                
                print(index, end = '\r')

                if math.isnan(valor) or valor == 0:
                    print("Inconsistência NaN gerada.")
                    novaLinha = [[codUni, coluna, 'NaN', categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                               
                if not (isinstance(valor, np.int32) or isinstance(valor, np.int64)):
                    print("Inconsistência de valor não inteiro gerada: " + str(type(valor)))
                    novaLinha = [[codUni, coluna, type(valor), categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )

    return resultado

def valida_inteiro(df_colunas):
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
            2.  co_simec
            42. cod_municipio
            47. vagas_ofertadas
            48. total_inscritos

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (2,42,47,48)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_inteiro( df_colunas )
        Out:
        ---------------------------------------------------------------------------------
        codigo_universal_linha | coluna                 | tipo_inconsistencia | categoria
        ---------------------------------------------------------------------------------
        33                     | co_simec               | 'Jorge'             |
        11                     | cod_municipio          | 147.784.412-78      |  
        03                     | vagas_ofertadas        | -                   |  
        1478                   | total_inscritos        | 'Brasileiro'        |  
                                                 
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
              
                if not (isinstance(valor, np.int32) or isinstance(valor, np.int64)):
                    novaLinha = [[codUni, coluna, type(valor), categoria]]
                    print("Inconsistência de valor não inteiro gerada: " + str(type(valor)))
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
   
    return resultado

def valida_texto_nao_nulo(df_colunas):
    """
        Função que fará a validação dos campos do tipo texto da base dados que
        são referentes às colunas da base de dados fornecida:
            03. unidade_ensino
            05. dependencia_adm
            06. sistema_ensino
            07. municipio
            13. ds_eixo_tecnologico
            15. no_curso
            18. tipo_curso
            20. ds_tipo_nivel
            22. nome_ciclo
            26. nome_aluno
            31. modalidade_pagto
            32. situacao_matricula
            36. tipo_oferta
            39. subdependencia
            43. modalidade_ensino
            54. status_ciclo_matricula
            55. nome_completo_instituicao
            56. sigla_instituicao
        
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função(3,5,6,7,13,15,18,20,22,26,31,32,36,39,43,54,55,56)


        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_inteiro( df_colunas )
        Out:
        ----------------------------------------------------------------------------------------
        codigo_universal_linha |       coluna                  | tipo_inconsistencia | categoria
        ----------------------------------------------------------------------------------------
                               | 03. unidade_ensino            |                     |   
                               | 05. dependencia_adm           |                     |
                               | 06. sistema_ensino            |                     |   
                               | 07. municipio                 |                     |   
                               | 13. ds_eixo_tecnologico       |                     |
                               | 15. no_curso                  |
                               | 18. tipo_curso                |
                               | 20. ds_tipo_nivel             |
                               | 22. nome_ciclo                |
                               | 26. nome_aluno                |
                               | 31. modalidade_pagto          |
                               | 32. situacao_matricula        |
                               | 36. tipo_oferta               |
                               | 39. subdependencia            |
                               | 43. modalidade_ensino         |
                               | 54. status_ciclo_matricula    |
                               | 55. nome_completo_instituicao |
                               | 56. sigla_instituicao         | 
      
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
              
                if valor == 0:
                       novaLinha = [[codUni, coluna, 'NaN', categoria]]
                       resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                if type(valor) != str:
                    novaLinha = [[codUni, coluna, type(valor), categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                    
                if not valor[0].isalpha():
                   novaLinha = [[codUni, coluna, valor, categoria]]
                   resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                  
   
    return resultado

def valida_estado(df_colunas):
    """
        Função que fará a validação dos campos do tipo texto da base dados que
        são referentes às colunas da base de dados fornecida:
            03.estado
            
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função(03)


        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_estado( df_colunas )
        Out:
        -----------------------------------------------------------------------------
        codigo_universal_linha | coluna             | tipo_inconsistencia | categoria
        -----------------------------------------------------------------------------
                               | estado             |                     |
                               |                    |                     |    
                           
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    # Lista da sigla dos estados brasileiros 
    sigla_estado = 'RO AC AM RR PA AP TO MA PI CE RN PB PE AL SE BA MG ES RJ SP PR SC RS MS MT GO DF'
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)

            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
              
                if valor == 0:
                    novaLinha = [[codUni, coluna, 'NaN', categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                       
                #Upper para que caso o valor seja Df ou df. Para que o dado padronize com a lista sigla_estado, pois assim não é uma incosistência    
                if not valor.upper() in sigla_estado:
                    novaLinha = [[codUni, coluna, valor, categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
   
    return resultado

def valida_sexo_aluno(df_colunas): 
    """
        Função que fará a validação dos campos do tipo texto da base dados que
        são referentes às colunas da base de dados fornecida:
            27. sexo_aluno
            
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função(27)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_sexo_auno( df_colunas )
        Out:
        --------------------------------------------------------------------------
        codigo_universal_linha | coluna          | tipo_inconsistencia | categoria
        --------------------------------------------------------------------------
        3                      | sexo            | NaN                 |
                               |                 |                     |     
      
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
               
                if valor == 0:
                       novaLinha = [[codUni, coluna, 'NaN', categoria]]
                       resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )      

                if valor.upper() != 'M' and valor.upper() != 'F': #A função upper para padronizar, pois caso seja m ou f. Não gerar incosistência
                    novaLinha = [[codUni, coluna, valor, categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                         
    return resultado


def valida_texto(df_colunas):
    """
        Função que fará a validação dos campos do tipo texto da base dados que
        são referentes às colunas da base de dados fornecida:
            25. programa_associado
            60. codigo_diploma_certificado
            62. nacionalidade
        
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função(25,60,62)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_texto(df_colunas )
        Out:
        -------------------------------------------------------------------------------------------
        codigo_universal_linha |          coluna                  | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------------------
        3                      |     programa_associado           |                     |    
                               |     codigo_diploma_certificado   |                     |
                               |     nacionalidade                |                     |    
      """ 
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
              
                if type(valor) != str:
                    novaLinha = [[codUni, coluna, type(valor), categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                    
                if not valor[0].isalpha():
                   novaLinha = [[codUni, coluna, valor, categoria]]
                   resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                  
   
    return resultado

def valida_email(df_colunas):
    """
        Função que fará a validação dos campos do tipo texto da base dados que
        são referentes às colunas da base de dados fornecida:
            . email_aluno
            
        
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função()

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_texto(df_colunas )
        Out:
        -----------------------------------------------------------------------------------
        codigo_universal_linha |          coluna          | tipo_inconsistencia | categoria
        -----------------------------------------------------------------------------------
        3                      |     email_aluno          |                     |    
                               |                          |                     |      
      """ 
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
                
                # Passando o email para a função que verifica se o e_mail é valido
                email_aluno = verifica_email(valor) 
                
                # Se a função retorna False. o Email é invalido
                if (email_aluno == False): 
                    novaLinha = [[codUni, 'email', 'E-mail invalido', categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True)
   
    return resultado
    

def valida_numero_cpf(df_colunas): # Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          cpf
          numero_cpf  

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (numero das colunas)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_numero_cpf(df_colunas)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |    numero_cpf        | cpf_invalido        |    
                   29987       |    numero_cpf        | cpf_invalido        |                                 
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")

    #for coluna in df_colunas.columns:
        #if coluna != 'codigo_universal_linha':
            #print(coluna)
    
    categoria = buscar_categoria('numero_cpf')

    for index, linha in df_colunas.iterrows():
        codUni = str(linha['codigo_universal_linha'])
        

        #print(linha.cpf) Estava testando se pegava o valor da linha cpf 
        if linha.cpf == 'COM CPF':
               cpf = linha.numero_cpf
               if(len(cpf) == 10): # Acrescentar o 0 nos cpf que possui 10 digitos.
                      cpf = cpf.zfill(11) # Fução que acrescenta o 0 Ao inicio do cpf que contém 10 digitos.
                      #print(cpf) Testando se estava adicionando os 0 nos cpf que possuí 10 digitos.
               #print(cpf, linha.cpf) Estava testando se estava pegando os cpf com a linha cpf marcada COM CPF

               if cpf == 0: #Para verificar se o valor é nulo
                  novaLinha = [[codUni, cpf, "NaN", categoria]]
                  resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
                  
               cpf_valido = valida_cpf(cpf) # Usando a função cpf_valido para verificar se o cpf informado é valido

               if (cpf_valido == False): # Caso for invalido retorna False e registra a incosistência
                  novaLinha = [[codUni, cpf, 'cpf_invalido', categoria]]
                  resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                          ignore_index = True )
    
    return resultado

def valida_bool_nao_nulo(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          21. experimental
          33. contrato_aprendizagem
          35. atestado_baixa_renda
          50. sg_etec
          52. sg_brasilpro 

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (21,33,35,50,52)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_bool_nao_nulo(df_colunas)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                               |                      |                     |
                               |                      |                     |                     
    """

    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")

    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
                
                if(valor == 'Nan' or valor == 'nan' or valor == 'None' or valor == 0):
                    novaLinha = [[codUni, valor, 'Nan', categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                                    ignore_index = True )
                
                # Passando os valores das linhas da coluna para a função que verifica se o valor é Não ou Sim    
                verifica_boleano_nao_nulo = verifica_boleano(valor) 
            
                if (verifica_boleano_nao_nulo == False): #Se a função retorna False. o valor boleano é invalido
                    novaLinha = [[codUni, valor, "Boleano Invalido", categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True)
   
    return resultado

def valida_bool(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          40. carga
          51. sg_proeja
        
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (40,51)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_bool(df_colunas)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                               |                      |                     |
                               |                      |                     |                                          
    """

    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                categoria = buscar_categoria(coluna)
                
                # Passando os valores das linhas da coluna para a função que verifica se o valor é Não ou Sim
                verifica_bol = verifica_boleano(valor) 
                
                # Se a função retorna False. o valor boleano é invalido
                if (verifica_bol == False): 
                    novaLinha = [[codUni, valor, "Boleano Invalido", categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True)
   
    return resultado

def valida_tipo_cota(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          34. tipo_cota
          
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (34)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_tipo_cota(df_colunas)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                               |                      |                     |
                               |                      |                     |                                             
    """
    
    # Respotas sobre a linha da coluna tipo_cota
    cota = 'SEM COTA sem cota COM COTA com cota SIM sim NAO NÃO nao não nan Nan none None' 
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            categoria = buscar_categoria(coluna)

            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]

                print('Linha inicio', valor)
                if not valor in cota:
                    novaLinha = [[codUni, coluna, valor, categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                        ignore_index = True )
   
    return resultado


def valida_dt_data_inicio(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          11.dt_data_inicio
           

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (11)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_dt_data_inicio(df_colunas)
        Out:
        ----------------------------------------------------------------------------
        codigo_universal_linha |     coluna        | tipo_inconsistencia | categoria
        ----------------------------------------------------------------------------
                   29991       |   dt_data_inicio  |                     |   
                   29987       |   dt_data_inicio  |                     |                    
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    categoria = buscar_categoria('dt_data_inicio')

    for index, linha in df_colunas.iterrows():
        codUni = str(linha['codigo_universal_linha'])

        # Pegando os valores da linha dt_data_inicio
        data_inicio = linha.dt_data_inicio
        
        # Pegando os valores da linha dt_deferimento ue 
        data_deferimento_ue = linha.dt_deferimento_ue

        print(data_inicio)

        if (data_inicio == 'Nan' or data_inicio == 'nan' 
              or data_inicio == 'None' or data_inicio == 0):
              novaLinha = [[codUni, data_inicio, 'Nan', categoria]]
              resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                            ignore_index = True )
          
        # Chamada da função verifica_data para verificar se a data_inicio é uma data valida
        data_inicio_valida = verifica_data_valida(data_inicio)
        data_deferimento_ue_valida = verifica_data_valida(data_deferimento_ue)
        
        # Para Gerar incosistencia caso a data nao seja valida e caso nao esteja no formato de uma data
        if (data_inicio_valida == False):
            # print(data_inicio)
            novaLinha = [[codUni, 'data_inicio', 'Data invalida', categoria]]
            resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                          ignore_index = True )
          
        # Convertendo as datas para o tipo date ultilizando a funçao verifica_data_valida, pois essa função retorna a data convertida
        # Pois, para comparar duas datas elas precisam está no formato date()
    
        # Comparando as datas.A função verifica_data_valida ela retorna a data convertida caso ela nao seja invalida.
        if (data_inicio_valida != False and data_deferimento_ue_valida != False):
            # Comparando as datas. De acordo com o document exploração de dados, a data_inicio nao pode ser inferior a data deferimento ue
            if (data_inicio_valida < data_deferimento_ue_valida):
                novaLinha = [[codUni, 'dt_data_inicio', 'data_inicio inferior a dt_deferimento_ue', categoria]]
                resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                            ignore_index = True )
    
    return resultado

def valida_dt_deferimento_curso(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          16.dt_deferimento_curso
           

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (16)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_dt_deferimento_curso(df_colunas)
        Out:
        ---------------------------------------------------------------------------------
        codigo_universal_linha |     coluna             | tipo_inconsistencia | categoria
        ---------------------------------------------------------------------------------
                   29991       |  dt_deferimento_curso  |                     |  
                   29987       |  dt_deferimento_curso  |                     |                    
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    categoria = buscar_categoria('dt_deferimento_curso')

    for index, linha in df_colunas.iterrows():
        codUni = str(linha['codigo_universal_linha'])

        #Pegando os valores da linha dt_data_inicio
        data_inicio = linha.dt_data_inicio
        
        #Pegando os valores da linha dt_deferimento_curso
        data_deferimento_ue_curso = linha.dt_deferimento_curso

        print(data_deferimento_ue_curso)

        if (data_deferimento_ue_curso == 'Nan' or data_deferimento_ue_curso == 'nan' or data_deferimento_ue_curso == 'None' or data_deferimento_ue_curso == 0):
            novaLinha = [[codUni, 'dt_deferimento_curso', 'NaN', categoria]]
            resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                          ignore_index = True )
          
        # Chamada da função verifica_data para verificar se a data_deferimento_ue_curso_valida é uma data valida
        data_inicio_valida = verifica_data_valida(data_inicio)
        data_deferimento_ue_curso_valida = verifica_data_valida(data_deferimento_ue_curso)
        
        # Para Gerar incosistencia caso a data nao seja valida e caso nao esteja no formato de uma data
        if (data_deferimento_ue_curso_valida == False):
            # print(data_deferimento_ue_curso_valida)
            novaLinha = [[codUni, 'dt_deferimento_curso', 'Data invalida', categoria]]
            resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                          ignore_index = True )
          
        # Convertendo as datas para o tipo date ultilizando a funçao verifica_data_valida, pois essa função retorna a data convertida
        # Pois, para comparar duas datas elas precisam está no formato date()
    
        # Comparando as datas.A função verifica_data_valida ela retorna a data convertida caso ela nao seja invalida.
        if (data_inicio_valida != False and data_deferimento_ue_curso_valida != False):
            #Comparando as datas. De acordo com o document exploração de dados, a data_deferimento_ue_curso_valida nao pode ser inferior a data_inicio 
            if (data_deferimento_ue_curso_valida < data_inicio_valida):
                novaLinha = [[codUni, 'dt_deferimento_curso', 'dt_deferimento_curso inferior a data_inicio', categoria]]
                resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                                  ignore_index = True )

    return resultado

def valida_dt_data_fim_previsto(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          12.valida_dt_data_fim_previsto
           

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (12)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_dt_data_fim_previsto(df_colunas)
        Out:
        ---------------------------------------------------------------------------------
        codigo_universal_linha |     coluna             | tipo_inconsistencia | categoria
        ---------------------------------------------------------------------------------
                   29991       |  dt_data_fim_previsto  |                     |  
                   29987       |  dt_data_fim_previsto  |                     |                    
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    categoria = buscar_categoria('dt_data_fim_previsto')

    for index, linha in df_colunas.iterrows():
        codUni = str(linha['codigo_universal_linha'])

        # Pegando os valores da linha dt_data_inicio
        data_inicio = linha.dt_data_inicio
        
        # Pegando os valores da linha dt_data_fim_previsto
        data_fim_previsto = linha.dt_data_fim_previsto

        print(data_fim_previsto)

        # Chamada da função verifica_data para verificar se a da linha dt_data_fim_previsto é uma data valida
        data_inicio_valida = verifica_data_valida(data_inicio)
        data_fim_previsto_valida = verifica_data_valida(data_fim_previsto)
        
        # Para Gerar incosistencia caso a data nao seja valida e caso nao esteja no formato de uma data
        if (data_fim_previsto_valida == False):
            #print(data_fim_previsto)
            novaLinha = [[codUni, 'dt_data_fim_previsto', 'Data invalida', categoria]]
            resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                          ignore_index = True )
          
        # Convertendo as datas para o tipo date ultilizando a funçao verifica_data_valida, pois essa função retorna a data convertida
        # Pois, para comparar duas datas elas precisam está no formato date()
    
        # Comparando as datas.A função verifica_data_valida ela retorna a data convertida caso ela nao seja invalida.
        if (data_inicio_valida != False and data_fim_previsto_valida != False):
            # Comparando as datas. De acordo com o document exploração de dados, a data_fim_prevista nao pode ser inferior a data_inicio 
            if (data_fim_previsto_valida < data_inicio_valida):
                novaLinha = [[codUni, 'dt_data_fim_previsto', 'dt_data_fim_previsto inferior a data_inicio', categoria]]
                resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                                  ignore_index = True )
        
    return resultado

def valida_data_nao_nulo(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          23. dt_cadastro_ciclo
          28. dt_nascimento_aluno
          29. dt_cadastro_aluno_sistema
          44. dt_ocorrencia_ciclo
          45. dt_ocorrencia_matricula
          46. data_ultima_alteracao
          49. valida_mes_de_ocorrencia
          4.  valida_dt_deferimento_ue
          30. valida_periodo_cadastro_matricula_aluno

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (23,28,29,44,45,46)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_dt_deferimento_curso(df_colunas)
        Out:
        ---------------------------------------------------------------------------------
        codigo_universal_linha |     coluna             | tipo_inconsistencia | categoria
        ---------------------------------------------------------------------------------
                   29991       |                        |                     |      
                   29987       |                        |                     |                    
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            categoria = buscar_categoria(coluna)

            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                print(valor)
              
                # if (valor == 'Nan' or valor == 'nan' or valor == 'None' or valor == 0):
                #     novaLinha = [[codUni, coluna, 'NaN', categoria]]
                #     resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                #                         ignore_index = True )
                    
                # # Fazendo a chamada da função que verifica se a data é valida              
                # valida_data_nao_nulo = verifica_data(valor)
                
                # if (valida_data_nao_nulo == False):
                #     novaLinha = [[codUni, coluna, 'Data invalida', categoria]]
                #     resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                #                                   ignore_index = True)

    return resultado


def valida_data(df_colunas): #Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          53. ano_do_convenio_brasilPro
          57. dt_data_finalizado
          58. dt_validacao_conclusao
          59. data_geracao_codigo_diplomaCertificado
          
        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (53,57,58,59)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_data(df_colunas)
        Out:
        ---------------------------------------------------------------------------------
        codigo_universal_linha |     coluna             | tipo_inconsistencia | categoria
        ---------------------------------------------------------------------------------
                   29991       |                        |                     |  
                   29987       |                        |                     |                    
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")
    for coluna in df_colunas.columns:
        if coluna != 'codigo_universal_linha':
            print(coluna)
            categoria = buscar_categoria(coluna)
            
            for index, linha in df_colunas[['codigo_universal_linha', coluna]].iterrows():
                codUni = str(linha['codigo_universal_linha'])
                valor = linha[coluna]
                    
                # Fazendo a chamada da função que verifica se a data é valida              
                valida_data = verifica_data_nulo(valor)
                
                if (valida_data == False):
                    novaLinha = [[codUni, coluna,'Data invalida', categoria]]
                    resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                                  ignore_index = True)

    return resultado


def valida_codigo_diploma_certificado(df_colunas): # Testando criar a função
    """
        Esta função deve retornar o resultado com o código único da linha, a coluna 
        analisada e o que foi encontrado de problema nos valores. Caso retorne, 
        vazio os dados estão sem problemas. Colunas a serem processadas:
          59. valida_data_geracao_codigo_diplomaCertificado
          60. valida_codigo_diploma_certificado

        Parameters
        ----------
        df_colunas : DataFrame
            Este parâmetro deve conter as colunas 'codigo_universal_linha' e as
            colunas a serem processadas por essa função (59,60)

        Returns
        -------
        DataFrame
            Deve retornar um dataframe com os dados da linha com problema sendo:
            codigo_universal_linha, coluna, tipo_inconsistencia

        Examples
        --------
        >>> valida_codigo_diploma_certificado(df_colunas)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |                      |                     |    
                   29987       |                      |                     |                                 
    """
    resultado = pd.DataFrame( columns = ['codigo_universal_linha', 'coluna', 'tipo_inconsistencia', 'categoria'] )
    print("\nProcessando as colunas...")

    #categoria = buscar_categoria('Categoria')

    for index, linha in df_colunas.iterrows():
        codUni = str(linha['codigo_universal_linha'])

        data_geracao_codigo_diploma_certificado = linha.codigo_diploma_certificado #guardado valor da linha
        
        if linha.codigo_diploma_certificado != 'Nan':
               print(linha.codigo_diploma_certificado) 
               print(linha.data_geracao_codigo_diploma_certificado)
               if linha.data_geracao_codigo_diploma_certificado == 'Nan':
                  novaLinha = [[codUni, data_geracao_codigo_diploma_certificado, 'codigo diploma existe, mas a data não','categoria']]
                  resultado = pd.concat( [pd.DataFrame(novaLinha, columns = resultado.columns), resultado], 
                                          ignore_index = True )
            
          
    return resultado

'''
    FIM: ↑  Funções para limpeza de cada coluna  ↑
'''

'''
    INICIO: ↓  Funções que auxiliam nos processos  ↓
'''
def imprimir_exemplo_informacao_base_dados(df_base_dados):
    """
        Função para mostrar os tipos de dados de cada coluna e extrair 
        um exemplo de informação

        Parameters
        ----------
        df_base_dados : DataFrame
            O parâmetro 'df_base_dados' de conter um DataFrame convertido da
            base de dados do SISTEC.

        Returns
        -------
            Esta função não possui retorno, apenas imprime no console.

        Examples
        --------
        >>> verificarTipoInformacaoBaseDados(df_base_dados)
        Out:
        ----------------------------------------------------
        Tipo        Nome da coluna          Exemplo
        ----------------------------------------------------
        int         co_unidade_ensino       1254
        int         co_simec                14
        str         unidade_ensino          senac
        ...
        ----------------------------------------------------    

    """
    template = "%-15s %-40s %s"
    print(template % ("Tipo", "Nome da coluna", "Exemplo"))
    print("-" * 100)
    for c in df_base_dados.columns:
        print(template % (df_base_dados[c].dtype, c, df_base_dados[c].iloc[1]) )


def analisar_colunas(df_base_dados):
    """
        Função para analisar cada coluna do DataFrame e resumir os tipos de
        dados encontrados em cada coluna.

        Parameters
        ----------
        df_base_dados : DataFrame
            O parâmetro 'df_base_dados' de conter um DataFrame convertido da
            base de dados do SISTEC.

        Returns
        -------
            Esta função não possui retorno, apenas imprime no console.

        Examples
        --------
        >>> analisarColunas(df_base_dados)

    """
    dfCounts = pd.DataFrame({'valor_unico':[], 'quantidade':[], 'coluna_analisada':[]})

    for c in df_base_dados.columns:
        temp = None
        temp = pd.DataFrame( df_base_dados[c].value_counts(dropna = False) )
        temp['coluna_analisada'] = c
        temp = temp.reset_index()
        temp.columns = ['valor_unico', 'quantidade', 'coluna_analisada']
        dfCounts = dfCounts.append(temp, ignore_index = True)
        print(df_base_dados[c].value_counts(dropna = False))
        print('-' * 50)
        print('\n\r')

    dfCounts = dfCounts[['coluna_analisada', 'valor_unico', 'quantidade']]
    print(dfCounts)


def buscar_categoria(coluna):
    colunaCategoria = {'co_unidade_ensino' : 'Instituição', 
                       'co_simec' : 'Matrícula', 
                       'unidade_ensino' : 'Instituição',
                       'dt_deferimento_ue' : 'Instituição',
                       'dependencia_adm' : 'Instituição', 
                       'sistema_ensino' : 'Instituição', 
                       'municipio' : 'Instituição', 
                       'estado' : 'Instituição',
                       'co_ciclo_matricula' : 'Oferta', 
                       'periodo' : 'Oferta', 
                       'dt_data_inicio' : 'Oferta',
                       'dt_data_fim_previsto' : 'Oferta', 
                       'ds_eixo_tecnologico' : 'Curso', 
                       'co_curso' : 'Curso', 
                       'no_curso' : 'Curso',
                       'dt_deferimento_curso' : 'Curso', 
                       'co_tipo_curso' : 'Curso', 
                       'tipo_curso' : 'Curso', 
                       'co_tipo_nivel' : 'Curso',
                       'ds_tipo_nivel' : 'Curso', 
                       'experimental' : 'Curso', 
                       'nome_ciclo' : 'Oferta', 
                       'dt_cadastro_ciclo' : 'Oferta',
                       'carga_horaria' : 'Oferta', 
                       'programa_associado' : 'Matrícula', 
                       'nome_aluno' : 'Matrícula', 
                       'sexo_aluno' : 'Matrícula',
                       'dt_nascimento_aluno' : 'Matrícula', 
                       'dt_cadastro_aluno_sistema' : 'Matrícula',
                       'periodo_cadastro_matricula_aluno' : 'Matrícula', 
                       'modalidade_pagto' : 'Matrícula',
                       'situacao_matricula' : 'Matrícula', 
                       'contrato_aprendizagem' : 'Matrícula', 
                       'tipo_cota' : 'Matrícula',
                       'atestado_baixa_renda' : 'Matrícula', 
                       'tipo_oferta' : 'Oferta', 
                       'cpf' : 'Matrícula', 
                       'numero_cpf' : 'Matrícula',
                       'subdependencia' : 'Instituição', 
                       'carga' : '-', 
                       'cod_municipio' : 'Instituição', 
                       'modalidade_ensino' : 'Curso',
                       'dt_ocorrencia_ciclo' : 'Oferta', 
                       'dt_ocorrencia_matricula' : 'Matrícula',
                       'data_ultima_alteracao' : 'Matrícula', 
                       'vagas_ofertadas' : 'Oferta', 
                       'total_inscritos' : 'Oferta',
                       'mes_de_ocorrencia' : 'Matrícula', 
                       'sg_etec' : 'Oferta', 
                       'sg_proeja' : 'Oferta', 
                       'sg_brasilpro' : 'Oferta', 
                       'ano_convenio_brasilpro' : 'Oferta', 
                       'status_ciclo_matricula' : 'Oferta',
                       'nome_completo_instituicao' : 'Instituição', 
                       'sigla_instituicao' : 'Instituição',
                       'dt_data_finalizado' : 'Oferta', 
                       'dt_validacao_conclusao' : 'Matrícula',
                       'data_geracao_codigo_diploma_certificado' : 'Matrícula (Diploma)', 
                       'codigo_diploma_certificado' : 'Matrícula (Diploma)',
                       'email_aluno' : 'Matrícula', 
                       'nacionalidade' : 'Matrícula'}

    return colunaCategoria[coluna]
 
def verifica_boleano(valor_boleano):
    """
        Função para validar se o valor da linha que contém o cpf da coluna numero_cpf é valido.

        Parameters
        ----------
        boleanos : valor da linha  das colunas 

            O parâmetro 'valor_boleano' deve conter os dados das linhas das colunas.......
            da base de dados do SISTEC.

        Returns
        -------
            Esta função possui retorno. False ou True.
            False se o boleano for invalido e True se o boleano for valido

        Examples
        --------
        >>> verifica_boleano_nao_nulo(valor_boleano)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |                      | False               |
                   29987       |                      | False               |
                             
    """
    bolean = 'SIM sim não nao NAO NÃO'
    #Verifica se o valor boleeano está correto
    if (valor_boleano in  bolean):
        return True
    else:
        return False


def verifica_email(email):
    """
        Função para validar se o valor da linha que contém o email_aluno é valido
        Parameters
        ----------
        email : valor da linha  da coluna email_aluno

            O parâmetro 'email' deve conter o email do aluno da base de dados do SISTEC.

        Returns
        -------
            Esta função possui retorno. False ou True.
            False se o email for invalido e True se o email for valido

        Examples
        --------
        >>> valida_email(email)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     email_aluno      | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |    numero_cpf        | False               |    
                                          
    """
    if (re.search(REGEX, email)):
        return True

    if (email == 'nan' or email == 'None'):
        return 1
    else:
        return False


def valida_cpf(cpf: str) -> bool:
    """
        Função para validar se o valor da linha que contém o cpf da coluna numero_cpf é valido.

        Parameters
        ----------
        cpf : valor da linha  da coluna numero_cpf

            O parâmetro 'cpf' deve conter o cpf da linha da coluna numero_cpf
            da base de dados do SISTEC.

        Returns
        -------
            Esta função possui retorno. False ou True.
            False se o cpf for invalido e True se o cpf for valido

        Examples
        --------
        >>> valida_numero_cpf(cpf)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |    numero_cpf        | False               |
                   29987       |    numero_cpf        | False               |    
                             
    """
    if len(cpf) != TAMANHO_CPF:
        return False

    if cpf in (c * TAMANHO_CPF for c in "1234567890"):
        return False

    cpf_reverso = cpf[::-1]
    for i in range(2, 0, -1):
        cpf_enumerado = enumerate(cpf_reverso[i:], start=2)
        dv_calculado = sum(map(lambda x: int(x[1]) * x[0], cpf_enumerado)) * 10 % 11
        if cpf_reverso[i - 1:i] != str(dv_calculado % 10):
            return False

    return True

from datetime import datetime
def verifica_data(data):
    """
        Função para validar se o valor da linha que contém o data da colunas ......

        Parameters
        ----------
        data : valor da linha  da coluna que possui das data no dataframe.......

            O parâmetro 'data' deve conter a data da linha da coluna ...... 
            da base de dados do SISTEC.

        Returns
        -------
            Esta função possui retorno. False ou True.
            False se o data for invalido e True se o data for valido

        Examples
        --------
        >>> verifica_data(data)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |    verifica_data     | True                |
                   29987       |    verifica_data     | True                |
                             
    """
    

    format = "%d/%m/%Y"
    res = True

    if (data == 'Nan' or data == 'nan' or data == 'None' or data == 0):
        return False
    try:
        res = bool(datetime.strptime(data, format))
    except ValueError:
        res = False

    if(res == False):
        return False
    else:
      # faz o split e transforma em números
      dia, mes, ano = map(int, data.split('/'))

      # mês ou ano inválido (só considera do ano 1 em diante), retorna False
      if mes < 1 or mes > 12 or ano <= 0:
          return False

      # verifica qual o último dia do mês
      if mes in (1, 3, 5, 7, 8, 10, 12):
          ultimo_dia = 31
      elif mes == 2:
          # verifica se é ano bissexto
          if (ano % 4 == 0) and (ano % 100 != 0 or ano % 400 == 0):
              ultimo_dia = 29
          else:
              ultimo_dia = 28
      else:
          ultimo_dia = 30

      # verifica se o dia é válido
      if dia < 1 or dia > ultimo_dia:
          return False

      return True

from datetime import datetime
def verifica_data_nulo(data):
    """
        Função para validar se o valor da linha que contém o data que podem ser nulos das colunas ......

        Parameters
        ----------
        data : valor da linha  da coluna que possui das data no dataframe.......

            O parâmetro 'data' deve conter a data da linha da coluna ...... 
            da base de dados do SISTEC.

        Returns
        -------
            Esta função possui retorno. False ou True.
            False se o data for invalido e True se o data for valido

        Examples
        --------
        >>> verifica_data_nulo(data)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |    verifica_data_nulo     | True           |
                   29987       |    verifica_data_nulo     | True           |
                             
    """
    #Verificação para as data que podem ter valores vazios
    

    format = "%d/%m/%Y"
    res = True

    if (data == 'Nan' or data == 'nan' or data == 'None' or data == 0):
        return True
    try:
        res = bool(datetime.strptime(data, format))
    except ValueError:
        res = False

    if(res == False):
        return False

    # faz o split e transforma em números
    dia, mes, ano = map(int, data.split('/'))

    # mês ou ano inválido (só considera do ano 1 em diante), retorna False
    if mes < 1 or mes > 12 or ano <= 0:
        return False

    # verifica qual o último dia do mês
    if mes in (1, 3, 5, 7, 8, 10, 12):
        ultimo_dia = 31
    elif mes == 2:
        # verifica se é ano bissexto
        if (ano % 4 == 0) and (ano % 100 != 0 or ano % 400 == 0):
            ultimo_dia = 29
        else:
            ultimo_dia = 28
    else:
        ultimo_dia = 30

    # verifica se o dia é válido
    if dia < 1 or dia > ultimo_dia:
        return False

    return True


def verifica_data_valida(data):
    """
        Função para validar se o valor da linha que contém o data da colunas ......

        Parameters
        ----------
        data : valor da linha  da coluna que possui das data no dataframe.......

            O parâmetro 'data' deve conter a data da linha da coluna ...... 
            da base de dados do SISTEC.

        Returns
        -------
            Esta função possui retorno. False ou True.
            False se o data for invalido e True se o data for valido

        Examples
        --------
        >>> verifica_data(data)
        Out:
        -------------------------------------------------------------------------------
        codigo_universal_linha |     coluna           | tipo_inconsistencia | categoria
        -------------------------------------------------------------------------------
                   29991       |    verifica_data     | True                |
                   29987       |    verifica_data     | True                |
                             
    """
    

    format = "%d/%m/%Y"
    res = True

    try:
        res = bool(datetime.strptime(data, format))
    except ValueError:
        res = False

    if(res == False):
        return False
    else:
      data = datetime.strptime(data, '%d/%m/%Y').date()
      return data

'''
    FIM: ↑  Funções que auxiliam nos processos  ↑
'''


'\n    FIM: ↑  Funções que auxiliam nos processos  ↑\n'